In [ ]:
import matlab.engine

eng = matlab.engine.start_matlab()

import pyvisa
import numpy as np
from time import sleep
import os
from scipy import signal as sig
import pandas as pd
from scipy.signal import square, hilbert
import scipy.io


def set_arb_wave(device, arb_sig, sampRate=10000, chan=1):
    ampl = 2 * np.amax(abs(arb_sig))

    if ampl > 20:
        ampl = 20
    elif ampl < 0.002:
        ampl = 0.002

    if sampRate > 60000000:
        sampRate = 60000000
    elif sampRate < 0.000001:
        sampRate = 0.000001

    if (not chan == 1) and (not chan == 2):
        chan = 1

    arb_sig = arb_sig / np.amax(abs(arb_sig)) * (ampl / 2)

    device.write(":SOUR" + str(chan) + ":APPL:ARB " + str(sampRate) + ',' + str(ampl))
    sleep(0.02)
    arbVisa = list()
    # this for loop changes the voltages to a list of integers from 0 to 16383
    for x in range(0, len(arb_sig)):
        arbVisa.append(int((arb_sig[x] + ampl / 2) / (ampl) * 16383))
    end_cnt = -1

    # this for loop pulls out 8000 samples at a time and sends in a binary stream
    for x in range(0, int((len(arbVisa) - 8001) / 8000 + 1)):
        device.write_binary_values(":SOUR" + str(chan) + ":TRAC:DATA:DAC16 VOLATILE,CON,",
                                   arbVisa[(((x + 1) * 8000) - 8000):((x + 1) * 8000)], datatype='H',
                                   is_big_endian=False)
        sleep(0.02)
        end_cnt = ((x + 1) * 8000)

    # this line sends the last set of samples when there are 8000 or less points left to send.
    device.write_binary_values(":SOUR" + str(chan) + ":TRAC:DATA:DAC16 VOLATILE,END,", arbVisa[end_cnt + 1:],
                               datatype='H', is_big_endian=False)
    device.write(":OUTP" + str(chan) + " ON")


rm = pyvisa.ResourceManager()
dev = rm.open_resource('USB0::0x1AB1::0x0642::DG1ZA204104796::INSTR')
A = 230 * np.sqrt(2)
f = 50.0
t = np.arange(0, 60, 1 / 50000)

def sygnal(A,f):
    return (u / rms) * 230
file_path='odksztalcenie_napiecia.csv'

if os.path.exists(file_path):
    df=pd.read_csv(file_path)

else:
    df=pd.DataFrame([])
nazwy_kolumn = ['THD_A','THD_f']
df2 = pd.DataFrame([], columns=nazwy_kolumn)

mc = [0.96, 0.92, 0.88, 0.84, 0.80]
for m in mc:
    u = np.sin(2 * np.pi * 50 * t);
    u = np.clip(u, -m, m)
    rms = np.sqrt(np.mean(u ** 2))


    # skalowanie wart. chwilowych CHROMA!!! WAZNE!!!
    u = sygnal(A, f) * 10 / 1.431 / 300

    set_arb_wave(dev, u, 50000)
    sleep(0.2)
    dev = rm.close()
    sleep(10)

    eng.pico_scope_manage(nargout=0)
    data = scipy.io.loadmat('wav')
    data1 = data.flatten()
    fs=20000

    h = hilbert(np.array(data1))

    amplituda = np.abs(h)
    faza = np.angle(h)
    czestotliwosc = (np.diff(faza) / (2.0 * np.pi) * fs)
    b_w_A = ((abs(A - amplituda)) / A) * 100
    b_w_A=b_w_A[::205]
    b_w_f= ((abs(f - czestotliwosc)) /  f) * 100
    b_w_f=b_w_f[::205]

    df3 = pd.DataFrame(zip(b_w_A, b_w_f), columns=nazwy_kolumn)
    df2 = pd.concat([df2, df3], ignore_index=True)


    if df.empty:
      df=df2
    else:
      df=df.merge(df2, left_index=True, right_index=True)

df.to_csv(file_path,index=False)

<ipython-input-2-8b9cadf5f3d0>:52: FutureWarning: Passing 'suffixes' which cause duplicate columns {'THD_A_x', 'THD_f_x'} in the result is deprecated and will raise a MergeError in a future version.
  df=df.merge(df2, left_index=True, right_index=True)
<ipython-input-2-8b9cadf5f3d0>:52: FutureWarning: Passing 'suffixes' which cause duplicate columns {'THD_A_x', 'THD_f_x'} in the result is deprecated and will raise a MergeError in a future version.
  df=df.merge(df2, left_index=True, right_index=True)
